# Segmenting and Clustering Neighborhoods in Toronto

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [216]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## 2. Download and Refine Dataset

Here is where i download the data and save it into a file


In [217]:
!wget -q -O 'toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Open the file and use Beautiful soup to read it, get the first table of the html and then for each element of the table i create a row for the table_data array after that i set the first row with the headers (the first row is a TH and not a TD so it had blank data)

In [218]:
with open("toronto_data.html") as html_doc:
    soup = BeautifulSoup(html_doc, 'lxml') 
    table_data = soup.find_all('table')[0] 
    table_data = [[cell.text.replace('\n', '') for cell in row("td")]
                         for row in table_data("tr")]
    table_data[0] = ["PostalCode","Borough","Neighborhood"]

using numpy i transform the array into a numpyarray and then in a data frame 

In [219]:
data = np.array(table_data)
df = pd.DataFrame({'PostalCode':data[1:,0],'Borough':data[1:,1],'Neighborhood':data[1:,2]})

not_assigned = 'Not assigned'

Now the cleanup process

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [220]:
df = df[df['Borough']  != not_assigned]

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [221]:
df['Neighborhood'].loc[df['Neighborhood'] == not_assigned] = df['Borough']

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [222]:
toronto_data = df.groupby(['PostalCode', 'Borough'],as_index=False).agg({'Neighborhood': lambda x : ', '.join(x) } ) 

Using the .shape method to print the number of rows of the dataframe.

In [223]:
toronto_data.reset_index(drop=True,inplace=True)
toronto_data.shape

(103, 3)